In [1]:
library(bnlearn)
library(parallel)

data_numeric = read.table("../data/prediction.train.numeric.warp.txt", header = TRUE, sep="\t", row.names=1, check.names=F, stringsAsFactors=F)
data_response = read.table("../data/prediction.train.response.warp.txt", header = TRUE, sep="\t", row.names=1, check.names=F, stringsAsFactors=F)
blacklist = read.table("../data/blacklist.txt", header=T, sep="\t", row.names=1)

In [2]:
data_binary = read.table("../data/prediction.train.binary.warp.txt", header = TRUE, sep="\t", row.names=1, check.names=F, stringsAsFactors=T)

In [3]:
data_numeric <- as.data.frame(lapply(data_numeric, as.numeric))
data_response <- as.data.frame(lapply(data_response, as.numeric))

In [4]:
# data = dedup(data_numeric, threshold=.95)

In [5]:
cnames = sample(colnames(data_numeric), 10)
# cnames = colnames(data_numeric)
# cnames_response = sample(colnames(data_response), 100)
cnames_response =colnames(data_response)

In [6]:
# dim(data)

In [7]:
data_small <- data_numeric[,cnames]
data_response <- data_response[,cnames_response]

In [8]:
cnames

[1] "food.L1_Grain_Product.L2_Grain_mixtures_frozen_plate_meals_soups.L3_Mixtures_mainly_grain_pasta_or_bread"                                 
 [2] "food.L1_Meat_Poultry_Fish_and_Mixtures.L2_Beef.L3_Beef_NS"                                                                                
 [3] "nutrients.TFAT"                                                                                                                           
 [4] "kegg.K16089"                                                                                                                              
 [5] "tax.k__Bacteria.p__Firmicutes_A.c__Clostridia.o__Oscillospirales.f__Ruminococcaceae.g__Faecalibacterium.s__Faecalibacterium_prausnitzii_D"
 [6] "kegg.K01811"                                                                                                                              
 [7] "kegg.K15923"                                                                                                                              
 [8] "nutrients.PROT"                                                                                                                           
 [9] "food.L1_Grain_Product.L2_Cakes_cookies_pies_pastries_bars.L3_Cookies"                                                                     
[10] "tax.k__Bacteria.p__Firmicutes_A.c__Clostridia.o__Lachnospirales.f__Lachnospiraceae.g__CAG_882.s__CAG_882_sp003486385"

In [9]:
data = cbind(dedup(data_small, threshold=.95), data_binary, data_response)

In [10]:
blacklist_small <- blacklist[blacklist$from %in% colnames(data),]
blacklist_small <- blacklist_small[blacklist_small$to %in% colnames(data),]

In [11]:
print(dim(blacklist_small))

[1] 3232    2


In [12]:
colnames(data)

[1] "food.L1_Grain_Product.L2_Grain_mixtures_frozen_plate_meals_soups.L3_Mixtures_mainly_grain_pasta_or_bread"                                                          
 [2] "food.L1_Meat_Poultry_Fish_and_Mixtures.L2_Beef.L3_Beef_NS"                                                                                                         
 [3] "nutrients.TFAT"                                                                                                                                                    
 [4] "kegg.K16089"                                                                                                                                                       
 [5] "tax.k__Bacteria.p__Firmicutes_A.c__Clostridia.o__Oscillospirales.f__Ruminococcaceae.g__Faecalibacterium.s__Faecalibacterium_prausnitzii_D"                         
 [6] "kegg.K01811"                                                                                                                                                       
 [7] "kegg.K15923"                                                                                                                                                       
 [8] "nutrients.PROT"                                                                                                                                                    
 [9] "food.L1_Grain_Product.L2_Cakes_cookies_pies_pastries_bars.L3_Cookies"                                                                                              
[10] "tax.k__Bacteria.p__Firmicutes_A.c__Clostridia.o__Lachnospirales.f__Lachnospiraceae.g__CAG_882.s__CAG_882_sp003486385"                                              
[11] "Gender"                                                                                                                                                            
[12] "Supplement"                                                                                                                                                        
[13] "day_plus_one.tax.k__Bacteria.p__Actinobacteriota.c__Actinomycetia.o__Actinomycetales.f__Bifidobacteriaceae.g__Bifidobacterium.s__Bifidobacterium_adolescentis"     
[14] "day_plus_one.tax.k__Bacteria.p__Actinobacteriota.c__Actinomycetia.o__Actinomycetales.f__Bifidobacteriaceae.g__Bifidobacterium.s__Bifidobacterium_pseudocatenulatum"
[15] "day_plus_one.tax.k__Bacteria.p__Actinobacteriota.c__Coriobacteriia.o__Coriobacteriales.f__Coriobacteriaceae.g__Collinsella.s__Collinsella_sp900541285"             
[16] "day_plus_one.tax.k__Bacteria.p__Actinobacteriota.c__Coriobacteriia.o__Coriobacteriales.f__Coriobacteriaceae.g__Collinsella.s__Collinsella_sp900544865"             
[17] "day_plus_one.tax.k__Bacteria.p__Actinobacteriota.c__Coriobacteriia.o__Coriobacteriales.f__Coriobacteriaceae.g__Collinsella.s__Collinsella_sp900555225"             
[18] "day_plus_one.tax.k__Bacteria.p__Actinobacteriota.c__Coriobacteriia.o__Coriobacteriales.f__Eggerthellaceae.g__Eggerthella.s__Eggerthella_lenta"                     
[19] "day_plus_one.tax.k__Bacteria.p__Actinobacteriota.c__Coriobacteriia.o__Coriobacteriales.f__Eggerthellaceae.g__Enteroscipio.s__Enteroscipio_rubneri"                 
[20] "day_plus_one.tax.k__Bacteria.p__Bacteroidota.c__Bacteroidia.o__Bacteroidales.f__Bacteroidaceae.g__Bacteroides.s__Bacteroides_caccae"                               
[21] "day_plus_one.tax.k__Bacteria.p__Bacteroidota.c__Bacteroidia.o__Bacteroidales.f__Bacteroidaceae.g__Bacteroides.s__Bacteroides_cellulosilyticus"                     
[22] "day_plus_one.tax.k__Bacteria.p__Bacteroidota.c__Bacteroidia.o__Bacteroidales.f__Bacteroidaceae.g__Bacteroides.s__Bacteroides_faecis"                               
[23] "day_plus_one.tax.k__Bacteria.p__Bacteroidota.c__Bacteroidia.o__Bacteroidales.f__Bacteroidaceae.g__Bacteroides.s__Bacteroides_ovatus"                               
[24] "day_plus_one.tax.k__Bacteria.p__Bacteroidota.c__Bacteroidia.o__Bacteroidales.f__Bacte

In [13]:
# blacklist_small

In [14]:
# blacklist_small

In [15]:
# data_small

In [16]:
# data_small = dedup(data_small, threshold=.95)

In [17]:
data

,food.L1_Grain_Product.L2_Grain_mixtures_frozen_plate_meals_soups.L3_Mixtures_mainly_grain_pasta_or_bread,food.L1_Meat_Poultry_Fish_and_Mixtures.L2_Beef.L3_Beef_NS,nutrients.TFAT,kegg.K16089,tax.k__Bacteria.p__Firmicutes_A.c__Clostridia.o__Oscillospirales.f__Ruminococcaceae.g__Faecalibacterium.s__Faecalibacterium_prausnitzii_D,kegg.K01811,kegg.K15923,nutrients.PROT,food.L1_Grain_Product.L2_Cakes_cookies_pies_pastries_bars.L3_Cookies,tax.k__Bacteria.p__Firmicutes_A.c__Clostridia.o__Lachnospirales.f__Lachnospiraceae.g__CAG_882.s__CAG_882_sp003486385,⋯,day_plus_one.tax.k__Bacteria.p__Firmicutes_A.c__Clostridia.o__Oscillospirales.f__Ruminococcaceae.g__Faecalibacterium.s__Faecalibacterium_prausnitzii_G,day_plus_one.tax.k__Bacteria.p__Firmicutes_A.c__Clostridia.o__Oscillospirales.f__Ruminococcaceae.g__Faecalibacterium.s__Faecalibacterium_sp900539945,day_plus_one.tax.k__Bacteria.p__Firmicutes_A.c__Clostridia_A.o__Christensenellales.f__CAG_74.g__UBA11524.s__UBA11524_sp000437595,day_plus_one.tax.k__Bacteria.p__Firmicutes_A.c__Clostridia_A.o__Christensenellales.f__CAG_917.g__CAG_349.s__CAG_349_sp003539515,day_plus_one.tax.k__Bacteria.p__Firmicutes_A.c__Clostridia_A.o__Christensenellales.f__CAG_917.g__CAG_475.s__CAG_475_sp000434435,day_plus_one.tax.k__Bacteria.p__Proteobacteria.c__Alphaproteobacteria.o__RF32.f__CAG_239.g__CAG_495.s__CAG_495_sp000436375,day_plus_one.tax.k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Burkholderiales.f__Burkholderiaceae.g__Parasutterella.s__Parasutterella_sp000980495,day_plus_one.tax.k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Burkholderiales.f__Burkholderiaceae.g__Sutterella.s__Sutterella_wadsworthensis_A,day_plus_one.tax.k__Bacteria.p__Verrucomicrobiota.c__Verrucomicrobiae.o__Verrucomicrobiales.f__Akkermansiaceae.g__Akkermansia.s__Akkermansia_muciniphila_B,day_plus_one.tax.other
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,-15.232420,-3.787154,179.02632,0.53088992,0.02801749,0.1420302,-0.19233366,149.19259,16.8853943,-19.0442324,⋯,2.593214,3.3677632,0.1275953,-11.671053,-11.6664121,-3.8397878,-0.1546931,-11.6661117,3.6783540,7.348045
1,8.922108,-3.128246,53.62548,-0.08714397,1.79222502,-0.2366709,-0.55472831,109.62499,-3.1255605,-3.8435111,⋯,4.833517,4.7709130,1.1041871,-2.865086,-11.3833305,-11.3828397,2.3811447,-11.3835159,3.2821685,8.488893
2,9.287620,-2.479028,50.55367,-1.45420745,3.41021707,-0.9753296,-1.09102599,89.99474,-2.4812272,-1.6130819,⋯,5.101719,6.4897026,0.7697353,-11.471327,-11.4731409,-11.4733175,0.1799849,-11.4726997,4.0076714,7.729978
3,-2.256250,-2.259087,92.01400,-0.59611354,4.37920849,-0.6515340,-0.94769722,94.60735,7.9299060,-3.6488048,⋯,5.153771,6.6138067,1.1156384,-13.765381,-11.5479976,-11.4810317,-0.3454619,-14.1600187,3.1648075,7.548797
4,-8.464660,-1.931560,102.07241,-0.30166467,4.45977789,-0.5363300,-0.83208678,123.77266,5.2882770,-3.3576906,⋯,5.340829,6.5126798,1.1940547,-11.521147,-11.5205743,-11.5207177,0.2546097,-11.5234857,2.1296109,7.730921
5,-2.066289,-2.061294,108.39105,-0.55789850,4.46676031,-0.6577683,-0.93243462,161.80578,-2.0600636,-2.5977779,⋯,5.515798,7.1254429,0.5270837,-11.390707,-11.3912904,-11.3906777,0.2615313,-3.5665788,2.6165814,7.758828
6,8.937385,-2.750149,122.48491,-0.70070423,4.79249053,-0.6933233,-1.10628579,172.09464,-2.7505185,-3.5669973,⋯,5.372617,6.1931112,1.4405957,-11.361735,-11.3598593,-11.3623969,0.1529291,-11.3592001,3.4356867,7.997441
7,-2.249670,-2.248965,73.90019,-0.54513292,4.21009393,-0.4532041,-0.87177114,108.63027,-2.2488556,-2.8434270,⋯,5.124023,5.7599552,1.1019475,-4.025227,-11.8460995,-4.0223560,-0.5870884,-4.0257476,2.6025426,7.488731
8,10.375042,-2.031581,85.21509,-0.60621848,3.96429132,-0.5133167,-0.96965631,84.28559,-2.0316119,-4.0190147,⋯,3.799364,4.8944770,0.9785960,-3.223067,-11.7436337,-11.7392945,-3.2260761,-3.9159442,1.6098783,7.322473


In [ ]:
learn.dag <- function() {
    cl = makeCluster(40, type = "SOCK")
    # dag = si.hiton.pc(data_small, cluster = cl, blacklist = blacklist_small, alpha = 0.05, test="mi-cg")
#     dag = si.hiton.pc(data_small, cluster = cl, blacklist = blacklist_small, alpha=0.05, undirected=T, test="cor")
    dag = si.hiton.pc(data, cluster = cl, blacklist = blacklist_small, alpha=0.05, undirected=T)
    # dag = si.hiton.pc(data_small, blacklist = blacklist_small, alpha = 0.05)
    on.exit(stopCluster(cl))
    return(dag)
}


start_time <- Sys.time()
dag <- learn.dag()
end_time <- Sys.time()
amat.data = amat(dag)
plot(dag)
write.table(amat.data, "../results/dag.prediction.train.response.warp.csv", sep="\t")
print(end_time - start_time)

In [ ]:
dag

In [ ]:
dag$nodes

In [ ]:
# blacklist